In [19]:

import os
import shutil
import zipfile
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from tqdm import tqdm
import glob

# --- CONFIGURATION ---
# Kaggle Paths
MONET_PATH = '../input/gan-getting-started/monet_jpg'
PHOTO_PATH = '../input/gan-getting-started/photo_jpg'

# --- DYNAMIC WEIGHTS SEARCH ---
# We search for ANY .pth file in ../input/
# This handles cases where the dataset name is different or files are nested.
print("Searching for weights in ../input/ ...")
found_weights = glob.glob('../input/**/*.pth', recursive=True)

WEIGHTS_PATH = None

# Prioritize files with 'Monet' and 'epoch' in the name if multiple exist
for f in found_weights:
    if 'G_Monet' in f and 'epoch' in f:
        WEIGHTS_PATH = f
        break

# Fallback: take the first .pth found if no specific match
if WEIGHTS_PATH is None and len(found_weights) > 0:
    WEIGHTS_PATH = found_weights[0]

if WEIGHTS_PATH:
    print(f"✅ Found weights at: {WEIGHTS_PATH}")
else:
    print("❌ No weights found! Please check your dataset.")
    # Default fallback for local testing only
    WEIGHTS_PATH = 'G_Monet_epoch_30_2.pth'

# Output
OUTPUT_DIR = '../tmp/images'
os.makedirs(OUTPUT_DIR, exist_ok=True)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")


Searching for weights in ../input/ ...
✅ Found weights at: ../input/monet-gan-weights/G_Photo_epoch_30_2.pth
Using device: cpu


In [20]:

# --- MODELS (Inlined from src/models/generator/resnet_gan.py) ---

class ResNetBlock(nn.Module):
    def __init__(self, dim):
        super(ResNetBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(dim, dim, kernel_size=3, padding=0),
            nn.InstanceNorm2d(dim),
            nn.ReLU(True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(dim, dim, kernel_size=3, padding=0),
            nn.InstanceNorm2d(dim)
        )

    def forward(self, x):
        return x + self.conv_block(x)

class ResNetGenerator(nn.Module):
    def __init__(self, input_nc=3, output_nc=3, ngf=32, n_blocks=6, n_downsampling=2):
        super(ResNetGenerator, self).__init__()
        
        # Initial convolution
        model = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(input_nc, ngf, kernel_size=7, padding=0),
            nn.InstanceNorm2d(ngf),
            nn.ReLU(True)
        ]

        # Downsampling
        in_features = ngf
        out_features = in_features * 2
        for _ in range(n_downsampling):
            model += [
                nn.Conv2d(in_features, out_features, kernel_size=3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(True)
            ]
            in_features = out_features
            out_features = in_features * 2

        # Residual blocks
        for _ in range(n_blocks):
            model += [ResNetBlock(in_features)]

        # Upsampling
        out_features = in_features // 2
        for _ in range(n_downsampling):
            model += [
                nn.ConvTranspose2d(in_features, out_features, kernel_size=3, stride=2, padding=1, output_padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(True)
            ]
            in_features = out_features
            out_features = in_features // 2

        # Output layer
        model += [
            nn.ReflectionPad2d(3),
            nn.Conv2d(ngf, output_nc, kernel_size=7, padding=0),
            nn.Tanh()
        ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


In [23]:

# --- DATASET & TRANSFORMS ---

class ImageDataset(Dataset):
    def __init__(self, files, transform=None):
        self.files = files
        self.transform = transform

    def __getitem__(self, index):
        img_path = self.files[index]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, os.path.basename(img_path)

    def __len__(self):
        return len(self.files)

def get_transforms():
    # Only resize and normalize for inference
    return transforms.Compose([
        transforms.Resize((256, 256), Image.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

def denormalize(tensor):
    return tensor * 0.5 + 0.5


In [24]:

# --- MAIN EXECUTION ---

# 1. Initialize Model
# Experiment 2 used defaults: ngf=32, n_blocks=6
model = ResNetGenerator(ngf=32, n_blocks=6).to(DEVICE)

# 2. Load Weights
# Fallback check for weights in current directory if dataset path fails
if not os.path.exists(WEIGHTS_PATH):
    if os.path.exists('G_Monet_epoch_30_2.pth'):
        WEIGHTS_PATH = 'G_Monet_epoch_30_2.pth'
        print(f"Found weights in current directory: {WEIGHTS_PATH}")
    else:
        print(f"⚠️ Weights not found at {WEIGHTS_PATH}. Please make sure you added the dataset correctly.")

if os.path.exists(WEIGHTS_PATH):
    print(f"Loading weights from {WEIGHTS_PATH}...")
    state_dict = torch.load(WEIGHTS_PATH, map_location=DEVICE)
    model.load_state_dict(state_dict)
else:
    raise FileNotFoundError(f"Could not find weight file at {WEIGHTS_PATH}")


print(f"Model initialized with ngf=32, n_blocks=6")
# print(model) # Uncomment to debug structure

model.eval()

# 3. Prepare Data
import glob
photo_files = sorted(glob.glob(os.path.join(PHOTO_PATH, "*.jpg")))
print(f"Found {len(photo_files)} photo images.")

# 4. Generate Images
dataset = ImageDataset(photo_files, transform=get_transforms())
loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

print("Starting generation...")
cnt = 0

with torch.no_grad():
    for img_tensor, filename in tqdm(loader):
        img_tensor = img_tensor.to(DEVICE)
        
        # Generate
        generated = model(img_tensor)
        
        # Post-process
        generated = denormalize(generated).cpu()
        
        # Save images
        # We need to save as jpg
        for i in range(generated.size(0)):
            img_np = generated[i].permute(1, 2, 0).numpy()
            img_np = (img_np * 255).astype(np.uint8)
            img_pil = Image.fromarray(img_np)
            
            # Save to temporary directory
            save_path = os.path.join(OUTPUT_DIR, filename[i])
            img_pil.save(save_path)
            cnt += 1

print(f"Generated {cnt} images at {OUTPUT_DIR}")


Loading weights from ../input/monet-gan-weights/G_Photo_epoch_30_2.pth...
Model initialized with ngf=32, n_blocks=6
Found 7038 photo images.
Starting generation...


100%|██████████| 7038/7038 [32:46<00:00,  3.58it/s]

Generated 7038 images at ../tmp/images


In [26]:

# --- ZIP RESULTS ---
import shutil

# The competition requires a file named 'images.zip' in the working directory
print("Zipping images...")
shutil.make_archive('images', 'zip', OUTPUT_DIR)
print("✅ Created images.zip")

# Verify
if os.path.exists("images.zip"):
    print(f"images.zip created successfully. Size: {os.path.getsize('images.zip') / 1024 / 1024:.2f} MB")
else:
    print("❌ Failed to create images.zip")

# Optional: Clean up temp folder to save space/inodes if needed
# shutil.rmtree(OUTPUT_DIR)


Zipping images...
✅ Created images.zip
images.zip created successfully. Size: 77.71 MB
